## Bias scan using Multi-Dimensional Subset Scan (MDSS)

"Identifying Significant Predictive Bias in Classifiers" https://arxiv.org/abs/1611.08292

The goal of bias scan is to identify a subgroup(s) that has significantly more predictive bias than would be expected from an unbiased classifier. There are $\prod_{m=1}^{M}\left(2^{|X_{m}|}-1\right)$ unique subgroups from a dataset with $M$ features, with each feature having $|X_{m}|$ discretized values, where a subgroup is any $M$-dimension
Cartesian set product, between subsets of feature-values from each feature --- excluding the empty set. Bias scan mitigates this computational hurdle by approximately identifing the most statistically biased subgroup in linear time (rather than exponential).


We define the statistical measure of predictive bias function, $score_{bias}(S)$ as a likelihood ratio score and a function of a given subgroup $S$. The null hypothesis is that the given prediction's odds are correct for all subgroups in

$\mathcal{D}$: $H_{0}:odds(y_{i})=\frac{\hat{p}_{i}}{1-\hat{p}_{i}}\ \forall i\in\mathcal{D}$.

The alternative hypothesis assumes some constant multiplicative bias in the odds for some given subgroup $S$:


$H_{1}:\ odds(y_{i})=q\frac{\hat{p}_{i}}{1-\hat{p}_{i}},\ \text{where}\ q>1\ \forall i\in S\ \mbox{and}\ q=1\ \forall i\notin S.$

In the classification setting, each observation's likelihood is Bernoulli distributed and assumed independent. This results in the following scoring function for a subgroup $S$

\begin{align*}
score_{bias}(S)= & \max_{q}\log\prod_{i\in S}\frac{Bernoulli(\frac{q\hat{p}_{i}}{1-\hat{p}_{i}+q\hat{p}_{i}})}{Bernoulli(\hat{p}_{i})}\\
= & \max_{q}\log(q)\sum_{i\in S}y_{i}-\sum_{i\in S}\log(1-\hat{p}_{i}+q\hat{p}_{i}).
\end{align*}
Our bias scan is thus represented as: $S^{*}=FSS(\mathcal{D},\mathcal{E},F_{score})=MDSS(\mathcal{D},\hat{p},score_{bias})$.

where $S^{*}$ is the detected most anomalous subgroup, $FSS$ is one of several subset scan algorithms for different problem settings, $\mathcal{D}$ is a dataset with outcomes $Y$ and discretized features $\mathcal{X}$, $\mathcal{E}$ are a set of expectations or 'normal' values for $Y$, and $F_{score}$ is an expectation-based scoring statistic that measures the amount of anomalousness between subgroup observations and their expectations.

Predictive bias emphasizes comparable predictions for a subgroup and its observations and Bias scan provides a more general method that can detect and characterize such bias, or poor classifier fit, in the larger space of all possible subgroups, without a priori specification.

In [1]:
import sys
import itertools
sys.path.append("../")

from aif360.sklearn.datasets import fetch_compas
from aif360.sklearn.metrics import mdss_bias_scan, mdss_bias_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder

from IPython.display import Markdown, display
import numpy as np
import pandas as pd

We'll demonstrate scoring a subset and finding the most anomalous subset with bias scan using the compas dataset.

We can specify subgroups to be scored or scan for the most anomalous subgroup. Bias scan allows us to decide if we aim to identify bias as `higher` than expected probabilities or `lower` than expected probabilities. Depending on the favourable label, the corresponding subgroup may be categorized as priviledged or unprivileged.

In [2]:
np.random.seed(0)

#load the data, reindex and change target class to 0/1
X, y = fetch_compas(usecols=['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree'])

X.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y.index = pd.MultiIndex.from_arrays(y.index.codes, names=y.index.names)

y = pd.Series(y.factorize(sort=True)[0], index=y.index)

# Quantize priors count between 0, 1-3, and >3
def quantize_priors_count(x):
    if x <= 0:
        return '0'
    elif 1 <= x <= 3:
        return '1 to 3'
    else:
        return 'More than 3'
    
X['priors_count'] = pd.Categorical(X['priors_count'].apply(lambda x: quantize_priors_count(x)),  ordered=True, categories=['0', '1 to 3', 'More than 3'])
enc = OrdinalEncoder()

X_vals = enc.fit_transform(X)

### training
We'll split the dataset and then train a simple classifier to predict the probability of the outcome

In [3]:
np.random.seed(0)

(X_train, X_test,
 y_train, y_test) = train_test_split(X_vals, y, train_size=0.7, random_state=1234567)

In [4]:
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2')
clf.fit(X_train, y_train)

LogisticRegression()

In [5]:
test_prob = clf.predict_proba(X_test)[:,1]

In [6]:
dff = pd.DataFrame(X_test, columns=X.columns)
dff['observed'] = pd.Series(y_test.values)
dff['probabilities'] = pd.Series(test_prob)

In [7]:
dff.head()

,sex,race,age_cat,priors_count,c_charge_degree,observed,probabilities
0,1.0,0.0,0.0,1.0,0.0,1,0.487700
1,1.0,2.0,1.0,0.0,0.0,0,0.322064
2,1.0,2.0,1.0,0.0,0.0,0,0.322064
3,1.0,2.0,0.0,2.0,0.0,0,0.643483
4,1.0,5.0,0.0,0.0,0.0,0,0.223666


### bias scoring

We'll call the MDSS Classification Metric and score the test set. The privileged argument indicates the direction for which to scan for bias depending on the positive label. In our case since the positive label is 0, `True` corresponds to checking for lower than expected probabilities and `False` corresponds to checking for higher than expected probabilities.

In [8]:
privileged_group = dff[dff['sex'] == 1]
unprivileged_group = dff[dff['sex'] == 0]

In [9]:
privileged_score = mdss_bias_score(privileged_group['observed'], privileged_group['probabilities'], \
                                   pos_label=0, privileged=True)
privileged_score

2.363262497629335

In [10]:
unprivileged_score = mdss_bias_score(unprivileged_group['observed'], unprivileged_group['probabilities'], \
                                     pos_label=0, privileged=False)
unprivileged_score

0.003755523381276868

In [11]:
assert privileged_score > 0
assert unprivileged_score > 0

### bias scan
We get the bias score for the apriori defined subgroup but assuming we had no prior knowledge 
about the predictive bias and wanted to find the subgroups with the most bias, we can apply bias scan to identify the priviledged and unpriviledged groups. The privileged argument is not a reference to a group but the direction for which to scan for bias.

In [12]:
privileged_subset = mdss_bias_scan(dff['observed'], dff['probabilities'], dataset = dff[dff.columns[:-2]], \
                                   pos_label=0, penalty=0.5, privileged=True)
unprivileged_subset = mdss_bias_scan(dff['observed'], dff['probabilities'], dataset = dff[dff.columns[:-2]], \
                                     pos_label=0, penalty=0.5, privileged=False)

In [13]:
print(privileged_subset)
print(unprivileged_subset)

({'age_cat': [1.0]}, 30.149019994560646)
({'sex': [1.0], 'age_cat': [2.0]}, 4.710934850314047)


In [14]:
assert privileged_subset[0]
assert unprivileged_subset[0]

We can observe that the bias score is higher than the score of the prior groups. These subgroups are guaranteed to be the highest scoring subgroup among the exponentially many subgroups according the LTSS property. 

For the purposes of this example, the logistic regression model systematically under estimates the recidivism risk of individuals belonging to the `Female` group whereas individuals belonging to the `Male`, and `age_cat=Less than 25` are assigned a higher risk that is actually observed. We refer to these subgroups as the `detected privileged group` and `detected unprivileged group` respectively.

As noted in the paper, predictive bias is different from predictive fairness so there's no the emphasis in the subgroups having comparable predictions between them. 
We can investigate the difference in what the model predicts vs what we actually observed as well as the multiplicative difference in the odds of the subgroups.

In [15]:
to_choose = dff[privileged_subset[0].keys()].isin(privileged_subset[0]).all(axis=1)
temp_df = dff.loc[to_choose]

In [16]:
"Our detected priviledged group has a size of {}, we observe {} as the mean outcome, but our model predicts {}"\
.format(len(temp_df), temp_df['observed'].mean(), temp_df['probabilities'].mean())

'Our detected priviledged group has a size of 403, we observe 0.2878411910669975 as the mean outcome, but our model predicts 0.4661469627631023'

In [17]:
group_obs = temp_df['observed'].mean()
group_prob = temp_df['probabilities'].mean()

odds_mul = (group_obs / (1 - group_obs)) / (group_prob /(1 - group_prob))
"This is a multiplicative decrease in the odds by {}"\
.format(odds_mul)

'This is a multiplicative decrease in the odds by 0.4628869654122457'

In [18]:
assert odds_mul < 1

In [19]:
to_choose = dff[unprivileged_subset[0].keys()].isin(unprivileged_subset[0]).all(axis=1)
temp_df = dff.loc[to_choose]

In [20]:
"Our detected unpriviledged group has a size of {}, we observe {} as the mean outcome, but our model predicts {}"\
.format(len(temp_df), temp_df['observed'].mean(), temp_df['probabilities'].mean())

'Our detected unpriviledged group has a size of 340, we observe 0.6147058823529412 as the mean outcome, but our model predicts 0.5271796434466836'

In [21]:
group_obs = temp_df['observed'].mean()
group_prob = temp_df['probabilities'].mean()

odds_mul = (group_obs / (1 - group_obs)) / (group_prob /(1 - group_prob))
"This is a multiplicative increase in the odds by {}"\
.format(odds_mul)

'This is a multiplicative increase in the odds by 1.430910678064278'

In [22]:
assert odds_mul > 1

In summary this notebook demonstrates the use of bias scan to identify subgroups with significant predictive bias, as quantified by a likelihood ratio score, using subset scannig. This allows consideration of not just subgroups of a priori interest or small dimensions, but the space of all possible subgroups of features.
It also presents opportunity for a kind of bias mitigation technique that uses the multiplicative odds in the over-or-under estimated subgroups to adjust for predictive fairness.